<a href="https://colab.research.google.com/github/FaustinoSachimuco/chatBot_Alinca_Democratica/blob/Faustino_Testes/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import re
import random

def extract_all_lines(file_path):
    lines = []
    with open(file_path, 'r') as script_file:
        print(f"Extraindo linhas de {file_path}...")
        lines.extend(script_file.readlines())
    return lines

def process_directory(directory_path):
    all_lines_list = []
    print(f"Processando diretório {directory_path}...")
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            all_lines_list.extend(extract_all_lines(file_path))
    return all_lines_list

# Defina o caminho do diretório que contém os arquivos de script
directory_path = "dados"

# Chame a função process_directory para processar os arquivos no diretório especificado
dialogs = process_directory(directory_path)

# Exemplo de como você pode imprimir a primeira linha coletada
print("Primeira linha coletada:")
print(dialogs[1])
print(len(dialogs))

Processando diretório dados...
Extraindo linhas de dados/texto.txt...
Primeira linha coletada:
 Apesar disso, nenhuma das alianças deve conseguir uma maioria absoluta para conseguir formar um governo.

5247


In [7]:
!pip install docarray
from docarray import BaseDoc
from docarray.typing import NdArray
embedding_dimensions = 128

class DialogLine(BaseDoc):
    text: str = ''
    embedding: NdArray[embedding_dimensions]

In [8]:
!pip install openai

In [9]:
from openai import OpenAI
client = OpenAI(api_key= "sk-proj-iaPc4pb5qLNeOyNYyn0UT3BlbkFJnypoQDWLoXjKdLkWWX5m")

embedding_model = "text-embedding-3-small"
response = client.embeddings.create(
    input= dialogs[1],
    dimensions=embedding_dimensions,
    model=embedding_model
)
print(response.data[0].embedding)

[-0.05508721247315407, 0.09779410809278488, 0.22118113934993744, 0.08780661970376968, 0.0746980533003807, 0.050405580550432205, -0.01784222200512886, -0.0235512126237154, -0.09472503513097763, -0.046270135790109634, -0.012191751971840858, -0.0384674146771431, -0.030820747837424278, 0.058936554938554764, 0.03776517137885094, -0.00566022377461195, -0.10164345055818558, -0.05019750818610191, 0.06429442018270493, -0.0053156036883592606, -0.06975632905960083, -0.09914657473564148, 0.03472210839390755, -0.036984898149967194, 0.12068209052085876, -0.04850691556930542, -0.02084627002477646, -0.0567517913877964, -0.006989937741309404, 0.08858689665794373, 0.1711396872997284, -0.025827007368206978, 0.02801176905632019, -0.014903197064995766, -0.1824796348810196, 0.19673261046409607, -0.012633905746042728, -0.13972072303295135, 0.011743095703423023, 0.0026789342518895864, 0.04900109022855759, 0.03230326622724533, -0.02887006849050522, 0.0694442167878151, 0.17550921440124512, -0.04257684946060181,

In [10]:
print(len(response.data[0].embedding))

128


In [11]:
embeddings = []

for i in range(0, len(dialogs), 128):
    dialog_slice = dialogs[i:i+128]
    slice_embeddings = client.embeddings.create(
        input = dialog_slice,
        dimensions=embedding_dimensions,
        model=embedding_model
    )

    embeddings.extend(slice_embeddings.data)

In [12]:
print(len(embeddings))

5247


In [13]:
!pip install vectordb

In [14]:
from docarray import DocList
import numpy as np
from vectordb import InMemoryExactNNVectorDB

In [15]:
db = InMemoryExactNNVectorDB[DialogLine](workspace = 'workspace')

WARNING - docarray - Index file does not exist: /content/workspace/InMemoryExactNNIndexer[DialogLine][DialogLineWithMatchesAndScores]/index.bin. Initializing empty InMemoryExactNNIndex.


In [16]:
doc_list = [DialogLine(text = dialogs[i], embedding=embeddings[i].embedding) for i in range(len(embeddings))]
db.index(inputs=DocList[DialogLine](doc_list))

<DocList[DialogLine] (length=5247)>

In [17]:
queryText = 'O que faz a alianca democratica'
response = client.embeddings.create(
    input=queryText,
    dimensions=embedding_dimensions,
    model=embedding_model
)
query = DialogLine(text=queryText, embedding=response.data[0].embedding)
results = db.search(inputs=DocList[DialogLine]([query]), limit=10)

for m in results[0].matches:
  print(m)

DialogLine(
    id='17afc73e3e280eeffcd4ac2a538e178b',
    text='da aliança democrática\n',
    embedding=NdArray([ 0.02143601,  0.04097069,  0.19822943,  0.12892273, -0.03787335,
          0.08703204, -0.00805   ,  0.18976542, -0.09690671, -0.09451471,
          0.02770735, -0.03453068, -0.01417567, -0.08684804,  0.04768669,
         -0.02252468, -0.10481872,  0.05265469,  0.11524539,  0.10678139,
         -0.17001608,  0.01899801, -0.04186002,  0.00151225,  0.09365605,
         -0.00125925, -0.02321468, -0.16645874,  0.06936803, -0.00121421,
          0.11230139, -0.05777603,  0.04943469,  0.007659  , -0.03057468,
          0.25710946, -0.08102138, -0.1514934 ,  0.0070495 , -0.01457434,
         -0.0729867 , -0.06182403,  0.04529469,  0.15100273,  0.20448543,
         -0.09819472,  0.00739834,  0.00987467, -0.02497801, -0.03040601,
          0.04940403,  0.0682027 ,  0.04241202,  0.11150406, -0.08930138,
         -0.11432539,  0.03655468, -0.03897735, -0.02833601,  0.12113339,
      

DialogLine(
    id='3e274e9b741f30d25768451463445b6e',
    text='A Aliança Democrática propõe, assim, uma “efetiva mudança política e de políticas” no país, “com muito 
mais ambição, para elevados níveis de prosperidade, de crescimento da economia e dos rendimentos e oportunidades 
para todos os portugueses”.\n',
    embedding=NdArray([ 0.06792788, -0.0150012 ,  0.13685308,  0.11025815,  0.04432488,
          0.11879069,  0.09640662,  0.13718551,  0.01233478, -0.01570763,
         -0.06244268, -0.03953226, -0.08344159, -0.1298719 ,  0.09568635,
          0.01534749, -0.04415866, -0.05141687,  0.1769671 , -0.00900349,
         -0.02671959, -0.07557393, -0.04684586,  0.08848355,  0.0535777 ,
         -0.08338619, -0.01156602, -0.19126187, -0.02818786, -0.02263339,
          0.15635602, -0.00890653,  0.05363311,  0.04296743, -0.03158148,
          0.18250771, -0.13175571,  0.02069418,  0.13386115, -0.13208815,
         -0.00362217, -0.05939534,  0.02004316,  0.03457341,  0.1034432 ,
     

DialogLine(
    id='f5fdfc294e2bb4ee4f563074f271b64c',
    text='Ambição reformista e justiça social é o que a Aliança Democrática tem para oferecer no seu programa 
eleitoral. Palavras de Luís Montenegro que, esta sexta-feira, apresentou o programa eleitoral da AD. Além disso, o 
líder do PSD disse querer "virar a página ao desespero".\n',
    embedding=NdArray([ 0.13571219, -0.00384319,  0.10883117,  0.13105206,  0.05916097,
          0.13059741,  0.00338144,  0.14832866,  0.02166678, -0.02595752,
          0.01292193, -0.20436393, -0.11565088, -0.10473935,  0.02416734,
         -0.03407013,  0.03949748, -0.02466461,  0.12696023,  0.0772332 ,
         -0.14389585, -0.05876315,  0.00715004,  0.04106033,  0.06649216,
         -0.06450307, -0.02452253, -0.12809685,  0.09070212,  0.09030431,
          0.12480066, -0.05239809,  0.06444624,  0.07899495, -0.03895758,
          0.1848141 , -0.08069988, -0.07683538,  0.07285722, -0.02573019,
         -0.06586701, -0.00627626,  0.0244515 ,  0.

DialogLine(
    id='5085d899e3dfb2d6c16c55971b1b561f',
    text='A Aliança Democrática quer unir Portugal\n',
    embedding=NdArray([-9.85326059e-03, -3.83205675e-02,  9.77622718e-02,
          8.53248909e-02,  1.88381448e-02,  1.51713595e-01,
         -3.39226685e-02,  1.73338950e-01, -3.03091053e-02,
         -8.77899602e-02, -1.10423744e-01, -5.10380678e-02,
         -3.55193578e-02, -6.08983301e-02,  5.79290465e-02,
         -2.68075913e-02, -7.56326988e-02, -6.94700377e-03,
          9.80423912e-02,  4.73684818e-02, -1.03196621e-01,
         -1.51041299e-01,  9.20197833e-03, -3.26866331e-03,
          7.62489662e-02, -5.12621626e-02,  7.84899369e-02,
         -1.06614098e-01, -6.95820823e-02, -2.20455322e-02,
          1.80958241e-01, -5.89935072e-02, -1.79137457e-02,
          1.60229281e-01, -5.59401885e-02,  1.61237717e-01,
         -8.33080187e-02,  4.10097316e-02, -3.98052111e-02,
         -3.54073085e-02, -7.50024291e-03, -6.02260418e-02,
          5.30269258e-02,  9.9050827

DialogLine(
    id='940bb96b54f4607e9ead186c50654c16',
    text='Presidente do PSD e líder da Aliança Democrática\n',
    embedding=NdArray([ 0.07633366, -0.14910473,  0.04374647,  0.15130514,  0.04337973,
          0.09273203,  0.07539061,  0.15224819, -0.08162514, -0.07015152,
          0.00654232, -0.08466382, -0.0580492 , -0.16461246,  0.00958755,
         -0.06412656, -0.09220812,  0.05000719,  0.13736914,  0.08738815,
         -0.10142893, -0.0598305 , -0.02855308,  0.05501053,  0.01714495,
         -0.06334069, -0.01283579, -0.14438954,  0.01705326,  0.00148168,
          0.05469618, -0.04615645,  0.02965329,  0.1178797 , -0.05810159,
          0.11117365, -0.12626226, -0.02603832, -0.04631362, -0.01129026,
          0.0395028 , -0.05595357,  0.02483332,  0.11651754,  0.20422003,
         -0.21050695,  0.07947712, -0.0528625 ,  0.08340643,  0.0168306 ,
          0.01663413,  0.11882274,  0.01697468,  0.15172428, -0.01556012,
         -0.05789203, -0.00053455, -0.14334172,  0.067

DialogLine(
    id='cea7bac2b48dba5b2aa220501a29e535',
    text='Hino da Aliança Democrática\n',
    embedding=NdArray([ 0.04951133, -0.06451729,  0.10124166,  0.06895528, -0.00517303,
          0.03861052, -0.05300625,  0.12126809, -0.0598574 , -0.05109237,
         -0.06729104, -0.01120593, -0.08066048, -0.02768197,  0.10900815,
          0.00963183, -0.02130236,  0.00781502,  0.06812315,  0.11167094,
         -0.06318589, -0.07633344, -0.06817863,  0.03691854,  0.05109237,
          0.06357422, -0.04005286, -0.13946386,  0.06939908, -0.07644439,
          0.1389091 , -0.06945455,  0.07150713,  0.04446312, -0.07566774,
          0.14545514, -0.13635726, -0.10362709, -0.04399158, -0.08859339,
          0.02597611, -0.03040024,  0.03508786,  0.06767936,  0.12426374,
         -0.11205927,  0.06301947, -0.04496239,  0.07888529, -0.06340779,
         -0.04790256,  0.13857625, -0.08054953,  0.12759224,  0.01700305,
         -0.08698462,  0.01650378, -0.03003965,  0.04823541, -0.04687628,
 

DialogLine(
    id='b75fcb3cfdd143314f61035da86975f6',
    text='Apenas a Aliança Democrática 2024 pode fazer de Portugal um País com Futuro. \n',
    embedding=NdArray([ 0.00822248,  0.00243684,  0.10650354,  0.14340004,  0.07295582,
          0.08748718, -0.02532524,  0.19662191, -0.01185532,  0.02746308,
          0.03414572, -0.01823895, -0.04487979, -0.04969368,  0.02427874,
         -0.07361362, -0.18322675, -0.03776361,  0.14088845,  0.0407536 ,
         -0.12641689, -0.10536734,  0.03012418, -0.05127838,  0.11577252,
         -0.02415914, -0.04807908, -0.10979253, -0.02574384,  0.09705516,
          0.17473517, -0.08485599, -0.02499634,  0.04715218, -0.0222904 ,
          0.22688064, -0.05707896, -0.02882353, -0.01238604, -0.09394557,
         -0.0390792 , -0.0777398 ,  0.01154137,  0.09041738,  0.18992433,
         -0.1233073 ,  0.01215432,  0.06572004,  0.00322359, -0.00219391,
          0.08868318,  0.09448377, -0.05722846,  0.2463754 , -0.09579936,
         -0.11080913, -0.

DialogLine(
    id='3a4cfd18c4b5b12726a8be17e73f29bc',
    text='Na “constituição da Aliança Democrática” é destacado que ambos os partidos políticos “valorizam e 
acolhem, entre outras, a iniciativa e as ideias do “Manifesto por uma Alternativa Reformista e Moderada“, que foi 
subscrito por mais de 100 personalidades da sociedade portuguesa e divulgado a 15 de dezembro deste ano. Não 
adiantam, contudo, nomes dessas mesmas individualidades.\n',
    embedding=NdArray([ 0.0741002 ,  0.00908426,  0.08585127,  0.04346756,  0.00625345,
          0.10758505,  0.02016507,  0.07695241,  0.01842522, -0.01966593,
         -0.01772643, -0.15949513, -0.08191524, -0.0043603 ,  0.04284007,
          0.0756404 , -0.13873109,  0.0813448 ,  0.22498168,  0.04523592,
         -0.17375614, -0.00332638, -0.04423765,  0.07951938,  0.02588373,
         -0.15573022, -0.10849775, -0.1615487 ,  0.05165337, -0.01205768,
          0.08533788, -0.01347665,  0.03194466,  0.11676914, -0.13211399,
          0.185735

DialogLine(
    id='df1ef25b5cb5f3d7728b9b09887e747d',
    text='Tendo participado na convenção do dia 21 Janeiro, é com muito gosto que apoio a Aliança Democrática, e o 
seu candidato a Primeiro Ministro, Dr. Luis Montenegro, para liderar uma Mudança Segura, para um Portugal, com mais
Ambição, Crescimento, e Prosperidade para todos, Jovens, Adultos e Idosos.\n',
    embedding=NdArray([ 0.11233201, -0.13712808,  0.10575816,  0.08217306,  0.01546872,
          0.08880457,  0.0815964 ,  0.2054038 , -0.08661328,  0.0055611 ,
         -0.02019727,  0.01045184, -0.05743464, -0.09399444,  0.03263856,
          0.04373913, -0.15661895,  0.00628912,  0.11613791,  0.11936717,
         -0.08257671, -0.02469517,  0.01920255,  0.02818392, -0.02832808,
         -0.14877647, -0.01432983, -0.19686934,  0.10431653, -0.03062028,
          0.11060204, -0.03883758, -0.08684395,  0.06193253, -0.03312872,
          0.02852991, -0.09312946, -0.07000568, -0.02505557, -0.10310556,
         -0.02309495, -0.015

DialogLine(
    id='8db697fb32eb59cf3b7df60a00951636',
    text='“Esta aliança promete aos portugueses uma maneira diferente de Governar”\n',
    embedding=NdArray([ 3.00610857e-03,  3.84954400e-02,  5.73262945e-02,
          1.67321488e-01, -5.11739254e-02,  6.56061172e-02,
         -2.36032531e-02,  1.09017707e-01, -2.94968802e-02,
         -6.29036799e-02, -4.25203554e-02, -7.90608376e-02,
          1.59846637e-02, -1.31557241e-01,  1.37882099e-01,
         -8.14757869e-02, -2.25395262e-01,  4.35265861e-02,
          2.03775719e-01, -7.83421006e-03, -9.74604487e-02,
         -6.07762225e-02,  7.12409988e-02,  4.71202582e-02,
          1.69333946e-02, -8.00958127e-02, -2.97268759e-02,
         -1.91930965e-01, -6.92285448e-02,  2.51269713e-02,
          1.86181083e-01, -3.19405794e-02,  8.59606937e-02,
          2.87062712e-02,  6.92285448e-02,  1.46161929e-01,
         -8.98706093e-02,  4.31240946e-02,  3.61379907e-02,
         -1.26452418e-04, -7.34834522e-02, -3.21130753e-02,
    

In [18]:
import openai

In [25]:
def generate_response(question):
  response = client.embeddings.create(
      input=question,
      dimensions=embedding_dimensions,
      model = embedding_model
  )
  query = DialogLine(text=queryText, embedding=response.data[0].embedding)
  results = db.search(inputs=DocList[DialogLine]([query]), limit=10)
  context = "\n"
  for result in results[0].matches:
    context += "\"" + result.text + "\"\n"
  prompt = f"Lt. commander Data is asked: '{question}' . How might respond? Take account Data's previous responses similar to this topic, listed here: {context}"
  print("PROMPT whith RAG: \n")
  print(prompt)

  print("\n RESPONSE: \n")
  completion = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are Lt. Cmdr. Data from AD"},
           {"role": "user", "content": prompt}
      ]
  )
  return (completion.choices[0].message.content)

In [30]:
print(generate_response("quero saber o que é a aliaca democratica"))

PROMPT whith RAG: 

Lt. commander Data is asked: 'quero saber o que é a aliaca democraticaquero saber o que é a aliaca democratica' . How might respond? Take account Data's previous responses similar to this topic, listed here: 
"Ambição reformista e justiça social é o que a Aliança Democrática tem para oferecer no seu programa eleitoral. Palavras de Luís Montenegro que, esta sexta-feira, apresentou o programa eleitoral da AD. Além disso, o líder do PSD disse querer "virar a página ao desespero".
"
"A Aliança Democrática propõe, assim, uma “efetiva mudança política e de políticas” no país, “com muito mais ambição, para elevados níveis de prosperidade, de crescimento da economia e dos rendimentos e oportunidades para todos os portugueses”.
"
"da aliança democrática
"
"Tendo participado na convenção do dia 21 Janeiro, é com muito gosto que apoio a Aliança Democrática, e o seu candidato a Primeiro Ministro, Dr. Luis Montenegro, para liderar uma Mudança Segura, para um Portugal, com mais A

In [32]:
print(generate_response("quem e o Pedro Nuno Santos"))

PROMPT whith RAG: 

Lt. commander Data is asked: 'quem e o Pedro Nuno Santosquem e o Pedro Nuno Santos' . How might respond? Take account Data's previous responses similar to this topic, listed here: 
" Sendo assim, Pedro Nuno Santos perde a maioria absoluta conquistada por António Costa, há dois anos.
"
"Luís Montenegro (PSD), Nuno Melo (CDS) e Gonçalo da Câmara Pereira (PPM) assinaram este domingo, numa sala com mais de duas dezenas de apoiantes, na Alfândega do Porto, o acordo que vai dar origem à nova Aliança Democrática (AD).
"
"António Leitão Amaro
"
"“Todos conhecemos a história do Pedro e do lobo mau, não faça como na história, tantas vezes acena com o papão, tantas vezes acena com o lobo mau, que qualquer dia já ninguém acredita em Pedro Nuno Santos”, criticou.Hugo Soares também aludiu à presença de António Costa na campanha do PS e disse que é o “momento de perguntar porque é que Pedro Nuno Santos, que não serviu para ministro, serviria para primeiro-ministro de Portugal”.
"


In [28]:
print(generate_response("Sou o Faustino?"))

PROMPT whith RAG: 

Lt. commander Data is asked: 'Sou o Faustino?Sou o Faustino?' . How might respond? Take account Data's previous responses similar to this topic, listed here: 
"Agora é hora de acreditar  
"
"Reformista, Humanista e Tecnológico. Eu voto AD, Força Luís Montenegro. 
"
"Por um Portugal com futuro. 
"
"Essa força p’ra lutar  
"
"Portugal merece ter FUTURO! 
"
"“Todos conhecemos a história do Pedro e do lobo mau, não faça como na história, tantas vezes acena com o papão, tantas vezes acena com o lobo mau, que qualquer dia já ninguém acredita em Pedro Nuno Santos”, criticou.Hugo Soares também aludiu à presença de António Costa na campanha do PS e disse que é o “momento de perguntar porque é que Pedro Nuno Santos, que não serviu para ministro, serviria para primeiro-ministro de Portugal”.
"
"Já não dá p’ra continuar assim
"
"É preciso mudar Portugal mais uma vez! Força AD, continuo a acreditar que é possível viver por cá e ver crescer as gerações seguintes sem saírem do paí

In [33]:
print(generate_response("qual o plano da Alinça Democratica para a Agricultura?"))

PROMPT whith RAG: 

Lt. commander Data is asked: 'qual o plano da Alinça Democratica para a Agricultura?qual o plano da Alinça Democratica para a Agricultura?' . How might respond? Take account Data's previous responses similar to this topic, listed here: 
"Falando já de medidas concretas, o social-democrata explicou que entre as prioridades da Aliança Democrática, está a redução dos impostos e uma reforma da economia. No caso da habitação, explicou Montenegro, está prevista a isenção de IMT na compra da primeira casa, a diminuição da burocracia e mais facilidade para conseguir o primeiro crédito.
"
"Os agricultores não foram esquecidos, nem a escassez de água, durante o discurso desta sexta-feira. A Aliança Democrática quer fomentar a agricultura como sector estratégico para melhorar as condições dos agricultores. Luís Montenegro diz que "é necessário desamarrar as pessoas de tanta burocracia".
"
"de uma agricultura moderna, competitiva e sustentável, definindo-se quatro objetivos 
"


In [34]:
print(generate_response("como isso seria feito?"))

PROMPT whith RAG: 

Lt. commander Data is asked: 'como isso seria feito?como isso seria feito?' . How might respond? Take account Data's previous responses similar to this topic, listed here: 
"De forma alinhada com estes princípios, será desejável adotar um modelo de procedi -
"
"adotando um programa de estímulos;
"
"Criar um mecanismo permanente de avaliação das pendências e celeridade pro -
"
"para tal efeito. 
"
"O IDE pode ser levado a cabo, na prática, por duas vias distintas, mas complementares: 
"
"garantir a sua qualidade, eficiência e facilidade de utilização, aplicando os princípios 
"
"análise e eventual proposta de caminhos e soluções, deve permitir a sua adoção e imple -
"
"Simplificar processos e criar uma via verde para os licenciamentos e investimen -
"
"Executar a Estratégia dos Biorresíduos (criando condições para que a recolha seleti -
"
"– aqui inverte-se o modus operandi , sendo o cidadão apenas responsável por ga -
"


 RESPONSE: 

Lt. Commander Data might respon

In [35]:
print(generate_response("porque devo votar na AD?"))

PROMPT whith RAG: 

Lt. commander Data is asked: 'porque devo votar na AD?porque devo votar na AD?' . How might respond? Take account Data's previous responses similar to this topic, listed here: 
"Votar na AD é escolher o caminho para a mudança.
"
"Voto na AD, porque se desejo que haja verdadeira democracia com futuro neste país, então só ha um caminho: votar na social democracia e democracia cristã...por isso voto em Luís Montenegro para liderar a mudança democrática. Voto AD!✌️
"
"Eu apoio e voto AD, com certeza!
"
"Voto na AD porque chega de um governo de esquerda que demonstrou incapacidade de governação. Voto AD porque apresenta o melhor programa eleitoral, apresenta um candidato do povo, dinâmico e com vontade de dar uma vida melhor aos portugueses. Voto Luis Montenegro.
"
"Eu VOTO AD.
"
"a fiabilidade, o voto secreto e pessoal poderá ajudar a combater a abstenção desses 
"
"Voto AD e em Luis Montenegro para conduzir o pais para a curva ascendente, onde todos são ouvidos, com po

In [38]:
print(generate_response("porque nao devo votar no AD?"))

PROMPT whith RAG: 

Lt. commander Data is asked: 'porque nao devo votar no AD?porque nao devo votar no AD?' . How might respond? Take account Data's previous responses similar to this topic, listed here: 
"Voto na AD, porque se desejo que haja verdadeira democracia com futuro neste país, então só ha um caminho: votar na social democracia e democracia cristã...por isso voto em Luís Montenegro para liderar a mudança democrática. Voto AD!✌️
"
"Votar na AD é escolher o caminho para a mudança.
"
"Eu apoio e voto AD, com certeza!
"
"Num almoço-comício na Trofa, perante 2.000 simpatizantes, o Presidente da Câmara do Porto, Rui Moreira, anunciou que votará na Aliança Democrática nas eleições legislativas de dia 10, porque “ser independente não é ser neutral”.“Meu caro Luís [Montenegro], nós não temos medo, o país não vai votar por medo. Eu, pela minha parte, já fiz a minha escolha: eu vou votar AD, espero que votem AD e que no dia 10 lhe possa telefonar a dar os parabéns”, disse.
"
"Eu VOTO AD